In [ ]:
!wget https://apt.repos.intel.com/openvino/2020/GPG-PUB-KEY-INTEL-OPENVINO-2020
!apt-key add GPG-PUB-KEY-INTEL-OPENVINO-2020
!touch /etc/apt/sources.list.d/intel-openvino-2020.list
!echo "deb https://apt.repos.intel.com/openvino/2020 all main" >> /etc/apt/sources.list.d/intel-openvino-2020.list

!apt update
!apt install intel-openvino-dev-ubuntu18-2020.4.287

!pip install test-generator==0.1.1
!source /opt/intel/openvino/bin/setupvars.sh

--2022-04-13 15:20:42--  https://apt.repos.intel.com/openvino/2020/GPG-PUB-KEY-INTEL-OPENVINO-2020
Resolving apt.repos.intel.com (apt.repos.intel.com)... 104.93.155.142, 2600:1406:5400:59d::4b23, 2600:1406:5400:5a9::4b23
Connecting to apt.repos.intel.com (apt.repos.intel.com)|104.93.155.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 939 [binary/octet-stream]
Saving to: ‘GPG-PUB-KEY-INTEL-OPENVINO-2020.1’

GPG-PUB-KEY-INTEL-O 100%[===================>]     939  --.-KB/s    in 0s      

2022-04-13 15:20:42 (236 MB/s) - ‘GPG-PUB-KEY-INTEL-OPENVINO-2020.1’ saved [939/939]

OK
Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://apt.repos.intel.com/openvino/2020 all InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda

In [ ]:
!pip install gdown
!gdown --id 1j9x9PX8YIrWXlv9GVE68c8jUiM8MexKA
!unzip mangodetection.zip
!mv /content/content/mangodetection /content/

Access denied with the following error:

 	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses. 

You may still be able to access the file from the browser:

	 https://drive.google.com/uc?id=1j9x9PX8YIrWXlv9GVE68c8jUiM8MexKA 

unzip:  cannot find or open mangodetection.zip, mangodetection.zip.zip or mangodetection.zip.ZIP.
mv: cannot stat '/content/content/mangodetection': No such file or directory


In [ ]:
import tensorflow as tf
import IPython.display as display
import PIL
import PIL.Image
import numpy as np
import matplotlib.pyplot as plt
import os
import time
import pathlib
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint

In [ ]:
data_dir = pathlib.Path('/content/drive/MyDrive/Mango (P0)')
test_dir = pathlib.Path('/content/drive/MyDrive/Mango (P0)')
train_count = len(list(data_dir.glob('*/*.JPG')))
test_count = len(list(test_dir.glob('*/*.JPG')))

batch_size = 32
img_height = 180
img_width = 180

#Generate More Data by Generator
image_gen_train = ImageDataGenerator(rescale = 1./255,validation_split=0.2,rotation_range = 45,width_shift_range=.15,height_shift_range =.15,horizontal_flip=True,zoom_range=0.5)

#Shuffle All Data 
train_data_gen = image_gen_train.flow_from_directory(batch_size=batch_size,subset = 'training',  directory= data_dir,shuffle=True,target_size=(img_height, img_width))  

#Divide Data for Validation
validation_data_gen = image_gen_train.flow_from_directory(batch_size=batch_size,subset = 'validation',  directory= data_dir,shuffle=True,target_size=(img_height, img_width)) 

CLASS_NAMES = np.array([item.name for item in data_dir.glob('*') if item.name != "LICENSE.txt"])

Found 3708 images belonging to 2 classes.
Found 927 images belonging to 2 classes.


In [ ]:
num_classes = 2
model = Sequential([
  Conv2D(16, 3, padding='same', activation='relu',input_shape=(img_height, img_width ,3)),
  MaxPooling2D(),
  Conv2D(32, 3, padding='same', activation='relu'),
  MaxPooling2D(),
  Conv2D(64, 3, padding='same', activation='relu'),
  MaxPooling2D(),
  Dropout(0.2),
  Flatten(),
  Dense(128, activation='relu'),
  Dense(num_classes)
])
model.compile(optimizer='adam',loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),metrics=['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 180, 180, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 90, 90, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 90, 90, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 45, 45, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 45, 45, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 22, 22, 64)       0

In [ ]:
rlr = ReduceLROnPlateau(patience=10, verbose=1)
es = EarlyStopping(patience=24, restore_best_weights=True, verbose=1)
mc = ModelCheckpoint('/content/mangodetection/model.h5', save_best_only=True, verbose=1)

epochs = 15
history = model.fit(
  train_data_gen,
  callbacks=[rlr, es, mc],  
  validation_data=validation_data_gen,
  epochs=epochs
)

Epoch 1/15
116/116 [==============================] - ETA: 0s - loss: 0.5244 - accuracy: 0.8029
Epoch 1: val_loss improved from inf to 0.23578, saving model to /content/mangodetection/model.h5
116/116 [==============================] - 1035s 9s/step - loss: 0.5244 - accuracy: 0.8029 - val_loss: 0.2358 - val_accuracy: 0.9417 - lr: 0.0010
Epoch 2/15
116/116 [==============================] - ETA: 0s - loss: 0.2200 - accuracy: 0.9067
Epoch 2: val_loss improved from 0.23578 to 0.09850, saving model to /content/mangodetection/model.h5
116/116 [==============================] - 283s 2s/step - loss: 0.2200 - accuracy: 0.9067 - val_loss: 0.0985 - val_accuracy: 0.9525 - lr: 0.0010
Epoch 3/15
116/116 [==============================] - ETA: 0s - loss: 0.1914 - accuracy: 0.9010
Epoch 3: val_loss did not improve from 0.09850
116/116 [==============================] - 281s 2s/step - loss: 0.1914 - accuracy: 0.9010 - val_loss: 0.1236 - val_accuracy: 0.9493 - lr: 0.0010
Epoch 4/15
116/116 [===========

In [ ]:
model.save('train_model.h5')

In [ ]:
from keras.models import load_model
new_model = load_model('/content/train_model.h5')

In [ ]:
new_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 180, 180, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 90, 90, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 90, 90, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 45, 45, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 45, 45, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 22, 22, 64)       0

In [ ]:
new_model.get_weights()

[array([[[[ 0.07728337, -0.07687469,  0.05950969,  0.13049279,
            0.11532191,  0.03884476,  0.02767301, -0.15973036,
            0.08188266, -0.20116955, -0.07272055, -0.03078886,
           -0.21750703,  0.04751497, -0.0423268 , -0.08881113],
          [ 0.05595689, -0.01542091, -0.14385045, -0.157823  ,
           -0.22745147,  0.06367639, -0.04611438, -0.16573162,
           -0.001161  , -0.19161382,  0.0354131 ,  0.14138485,
           -0.15251681,  0.1630985 ,  0.03695187, -0.1892011 ],
          [ 0.10001913,  0.07518502,  0.00129885, -0.0556021 ,
            0.00807542,  0.09123974, -0.18153442, -0.18795346,
           -0.17424119,  0.17836691, -0.17036374, -0.18187618,
            0.06972457, -0.06037201,  0.02792027, -0.15532021]],
 
         [[ 0.15874681, -0.15362264,  0.07751098, -0.0902717 ,
            0.04264965,  0.03607062, -0.12158833,  0.09572629,
            0.14326361,  0.06017868,  0.12200585, -0.15127452,
           -0.12412648,  0.01321864, -0.03529139,

In [ ]:
new_model.optimizer

In [ ]:
model_mango = load_model('/content/train_model.h5')

image_gen_test = ImageDataGenerator(rescale = 1./255)
test_data_gen = image_gen_test.flow_from_directory(batch_size=32,directory= test_dir,shuffle=True,target_size=(img_height, img_width))  

start_time = time.time()
predictions = model_mango.predict(test_data_gen)
for i in range(len(predictions)):
    if(predictions[i][0]<0):
      print("Healthy     "+str(abs(predictions[i][0]))+"%")
    else:  
      print("Diseased    "+str(100-abs(predictions[i][0]))+"%")
print("Total Time: ", time.time()-start_time)
print("Average Time Per Image: ", (time.time()-start_time)/len(predictions))

Found 4635 images belonging to 2 classes.
Diseased    91.01714706420898%
Diseased    91.83929061889648%
Healthy     2.055085%
Healthy     7.4281807%
Diseased    91.26692199707031%
Healthy     23.24353%
Diseased    89.32893943786621%
Diseased    90.22370529174805%
Diseased    89.47287654876709%
Healthy     5.137966%
Healthy     23.89822%
Diseased    89.96578788757324%
Diseased    88.76195049285889%
Diseased    91.31888484954834%
Diseased    90.7311019897461%
Diseased    92.93868017196655%
Diseased    91.7653923034668%
Healthy     7.2730513%
Healthy     8.335562%
Healthy     13.771359%
Diseased    91.91746997833252%
Healthy     13.136296%
Diseased    91.13321876525879%
Diseased    89.09537315368652%
Diseased    87.94946765899658%
Healthy     22.113548%
Diseased    92.79686212539673%
Diseased    98.98882746696472%
Diseased    92.73795127868652%
Diseased    94.26139116287231%
Healthy     12.197942%
Diseased    91.49127674102783%
Diseased    90.3939733505249%
Healthy     11.625613%
Diseased

In [ ]:
print('train_model.h5')

train_model.h5


In [ ]:
acc = new_model.evaluate_generator(val)[1]

print (f"accuracy = {acc*100}%")

NameError: ignored

In [ ]:
def predictions(path):
img = load_img(path,target_size=(256,256))
i = img_to_array(img)
im = preprocess_input(i)
img = np.expand_dims(im , axis= 0)
pred = np.argmax(model.prediction(img))
print(f"imagenbf {ref[pred]} ")

IndentationError: ignored

In [ ]:
train.CLASS_NAMES

NameError: ignored

In [ ]:
path="/content/drive/MyDrive/mango train/test/healthy/0001_0051.JPG"
predictions(path)

NameError: ignored